University of Michigan - ROB 101 Computational Linear Algebra

## Computing numerical derivatives

- The input is a vector such as $x \in \mathbb{R}^m$. If we use the standard basis of $\mathbb{R}^m$, we have
    $$x = x_1 e_1 + x_2 e_2 + \dots + x_m e_m = \sum_{i=1}^m x_i e_i.$$
- Then we can use a finite difference approximation to compute each column of $A = \begin{bmatrix} a^{\mathrm{col}}_1 & \dots & a^{\mathrm{col}}_m \end{bmatrix}$ as
    $$\boxed{a^{\mathrm{col}}_i = \frac{\partial f(x_0)}{\partial x_i} = \frac{f(x_0 + h e_i) - f(x_0 -h e_i)}{2h}}$$

- For the general case of $f:\mathbb{R}^m \to \mathbb{R}^n$, $A$ is an $n\times m$ matrix and called the Jacobian of $f$, i.e., $$A_{n\times m} = \begin{bmatrix} a^{\mathrm{col}}_1 & \dots & a^{\mathrm{col}}_m \end{bmatrix} = \frac{\partial f(x)}{\partial x}.$$
- Each column of the Jacobian $a^{\mathrm{col}}_i = \frac{\partial f(x)}{\partial x_i} \in \mathbb{R}^n$ shows the rate of change of $f$ along $e_i$.

In [1]:
using LinearAlgebra

function Jacobian(func, x0, h = 0.001) 
    # Numerical Jacobian of f:R^m -> R^n
    m = length(x0);  # Domain dimension
    f0 = func(x0); 
    n = length(f0); # Range dimension
    
    if m == 1 # f:R -> R^n
        return (func(x0 .+ h) .- func(x0 .- h)) ./ (2*h)
    else
        Im = Matrix(1.0I, m, m); # Create standard basis for I_m
        A = zeros(n,m); # Create Jacobian matrix
        # Compute and fill in the columns of the Jacobian using central differences
        for i = 1:m
            ei = Im[:,i:i]
            A[:,i] = (func(x0 + h*ei) - func(x0 - h*ei)) / (2*h);
        end
        return A
    end
end

Jacobian (generic function with 2 methods)

In [4]:
# Test block for the Jacobian function 

function f(x) # f(x) = x^T H x
    H = [2 -1; -1 2];
    return x' * H * x
end

function exact_grad(x) # exact gradient! use this http://www.matrixcalculus.org/
    H = [2 -1; -1 2];
   return (2*H*x)'
end


myfunc = f;
x0 = rand(2) # initial guess, x0
@show A = Jacobian(myfunc, x0)

# We know the true gradient!
@show df = exact_grad(x0)

Error = A - df; # Error is the difference between our numerical and true gradient (from calculus)
println("Error = ", norm(Error))

A = Jacobian(myfunc, x0) = [1.7122465081962357 -0.07788596347582666]
df = exact_grad(x0) = [1.7122465081962135 -0.0778859634758331]
Error = 2.3119311561848286e-14


## Example 1: $f:\mathbb{R} \to \mathbb{R}$

We want to compute the derivative $f(x) = \cos(x)$ at $x = \frac{\pi}{6}$. Here the function $f$ maps a real scalar to a real scalar $f:\mathbb{R} \to \mathbb{R}$. 

In [5]:
function f1(x) # f(x) = cos(x)
    return cos(x)
end


myfunc = f1;
x0 = π/6 # initial guess, x0
println("A: ", Jacobian(myfunc, x0))
println("True derivative: ", -sin(x0))

A: -0.4999999166667157
True derivative: -0.49999999999999994


## Example 2: $f:\mathbb{R} \to \mathbb{R}^2$

We want to compute the derivative $f(x) = \begin{bmatrix} \cos(x) \\ \sin(x) \end{bmatrix}$ at $x = \frac{\pi}{6}$. Here the function $f$ maps a real scalar to a 2-vector $f:\mathbb{R} \to \mathbb{R}^2$.  

In [4]:
function f2(x) # f:R -> R^2
    return [cos(x); sin(x)]
end

myfunc = f2;
x0 = π/6 # initial guess, x0
A = Jacobian(myfunc, x0);
println("A: \t\t", A)
println("True derivative: \t", [-sin(x0); cos(x0)])
size(A)

A: 		[-0.4999999166667157, 0.8660252594468454]
True derivative: 	[-0.49999999999999994, 0.8660254037844387]


(2,)

## Example 3: $f:\mathbb{R}^3 \to \mathbb{R}^3$

For the function 
 \begin{equation*}
 f(x_1,x_2,x_3):= 
 \left[
\begin{array}{c}
x_1 x_2 x_3  \\
\log(2+\cos(x_1)) + x_2^{x_1} \\
 \frac{x_1 x_3}{1+ x_2^2} \\
\end{array}
\right],
 \end{equation*}
 compute its Jacobian at the point 
 $$x_0 = \left[
\begin{array}{c}
\pi \\
1.0 \\
2.0 \\
\end{array}
\right].$$

In [5]:
function f3(x) 
    return [x[1]*x[2]*x[3]; log(2+cos(x[1])) + x[2]^x[1]; (x[1]*x[3] / (1+x[2]^2))]
end

myfunc = f3;
x0 = [π; 1.0; 2.0] # initial guess, x0
A = Jacobian(myfunc, x0)

3×3 Array{Float64,2}:
 2.0   6.28319  3.14159
 0.0   3.14159  0.0
 1.0  -3.14159  1.5708

## Example 4: $f:\mathbb{R}^3 \to \mathbb{R}^2$

For the function 
 \begin{equation*}
 f(x_1,x_2,x_3):= 
 \left[
\begin{array}{c}
x_1 x_2 x_3  \\
\log(2+\cos(x_1)) + x_2^{x_1} 
\end{array}
\right],
 \end{equation*}
 compute its Jacobian at the point 
 $$x_0 = \left[
\begin{array}{c}
\pi \\
1.0 \\
2.0
\end{array}
\right].$$

In [6]:
function f4(x) 
    return [x[1]*x[2]*x[3]; log(2+cos(x[1])) + x[2]^x[1]]
end

myfunc = f4;
x0 = [π; 1.0; 2.0] # initial guess, x0
A = Jacobian(myfunc, x0)

2×3 Array{Float64,2}:
 2.0  6.28319  3.14159
 0.0  3.14159  0.0